In [1]:
import pandas as pd

In [15]:
from psycopg2 import Error

In [22]:
from sqlalchemy import create_engine

In [3]:
pip install boto3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 179.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 264.9 kB/s eta 0:00:0000:0100:02�━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━ 7.5/11.0 MB 242.2 kB/s eta 0:00:15�━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 7.6/11.0 MB 217.0 kB/s eta 0:00:16�━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 7.7/11.0 MB 204.6 kB/s eta 0:00:17
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 625.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "d2b-internal-assessment-bucket"
response = s3.list_objects(Bucket=bucket_name, Prefix="orders_data")
# for example to download the orders.csv
s3.download_file(bucket_name, "orders_data/orders.csv", "orders.csv")

In [5]:
# to download the reviews.csv
s3.download_file(bucket_name, "orders_data/reviews.csv","reviews.csv")

In [6]:
# to download the shipments_deliveries.csv
s3.download_file(bucket_name, "orders_data/shipment_deliveries.csv", "shipments_deliveries.csv")

In [7]:
orders = pd.read_csv("orders.csv")
reviews = pd.read_csv("reviews.csv")
shipments_deliveries = pd.read_csv("shipments_deliveries.csv")

In [8]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
try:
    # Replace the connection details with your own
    connection = psycopg2.connect(
        host="localhost",
        database="autochek_db",
        user="auto",
        password="auto"
    )
    cursor = connection.cursor()
    print("Successfully connected to the PostgreSQL database")

except (Exception, Error) as e:
    print("Error while connecting to the PostgreSQL database:", e)

Successfully connected to the PostgreSQL database


In [20]:
# SQL statements to create the tables
create_table_queries = [

    '''CREATE TABLE IF NOT EXISTS orders(order_id integer PRIMARY KEY,
        customer_id integer NOT NULL,
        order_date date NOT NULL,
        product_id varchar NOT NULL,
        unit_price integer NOT NULL,
        quantity integer NOT NULL,
        amount int NOT NULL)
    ''',
    '''CREATE TABLE IF NOT EXISTS shipments_deliveries(shipment_id integer PRIMARY KEY,
        order_id integer NOT NULL,
        shipment_date date NULL,
        delivery_date date NULL)
    ''',
    '''CREATE TABLE IF NOT EXISTS reviews(review integer NOT NULL,
        PRODUCT_id integer NOT NULL)
    '''
]
try:
    # Execute the create table queries
    for query in create_table_queries:
        cursor.execute(query)
    connection.commit()
    print("Tables created successfully")

except (Exception, Error) as e:
    print("Error while creating tables:", e)
connection.commit()

Tables created successfully


In [23]:
# Create a SQLAlchemy engine using the connection
engine = create_engine('postgresql+psycopg2://', creator=lambda: connection)

# Use the engine to load data into tables
try:
    
    orders.to_sql('Orders_table', con=engine, if_exists='replace', index=False)
    reviews.to_sql('Reviews_table', con=engine, if_exists='replace', index=False)
    shipments_deliveries.to_sql('Shipments_Table', con=engine, if_exists='replace', index=False)
        
    print("Data loaded into the tables successfully")

except (Exception, Error) as e:
    print("Error while loading data into tables:", e)

Data loaded into the tables successfully
